### Monte-Carlo-Simulation und Export der Ergebnisse
Für vorgegebene Erkrankungsraten (15\,\%, 30\,\%, 50\,\% und 70\,\%) werden die zentralen Modellparameter (Konsultationsanteil, Hospitalisierungsrate, Letalität) als Dreiecksverteilungen simuliert (Monte-Carlo-Simulation).

Die Erkrankungsraten sowie die zentralen Modellparameter können verändert werden.

Die Simulationsergebnisse werden in einer EXCEL-Datei gespeichert 

>**Wichtig:** Die Datei muss sich in diesem Verzeichnis befinden. Dann werden die Daten als Reiter in diese  Datei importiert. Der Dateiname kann geändert werden, z. B.: filename="Pandemieplan_RKS_exakt_MC_export.xlsx" 

---
An diesen Stellen im Programm können Änderungen vorgenommen werden:
```
#================ Eingabe ======================
pop = 100_000    # Normierung, nicht ändern
n = 200_000      # Anzahl der MC-Simulationen
seed = 42        # oder None - für die (Pseudo)-Zufallszahlen

# ---------  Erkrankungsraten ------------------ 
attack_rates = [0.15, 0.30, 0.50, 0.70]

# ---------  Min, Wert von Meltzer, Max --------
tri = dict(
    consult_frac=(0.45, 0.50, 0.55),
    hosp_per_consult=(0.024, 0.027, 0.03),
    death_per_consult=(0.004, 0.007, 0.010),
)

# -- Name der EXCEL-Datei, in der die Daten exportiert werden --
# -- Diese Datei muss sich in diesem Ordner befinden -----------

filename="Pandemieplan_RKS_exakt_MC_export.xlsx"
#================================================
```
---


### ▶️ Ausführen des Programms

- Klicken Sie in die Code-Zelle.
- Drücken Sie `Shift + Enter`, um die Berechnung zu starten.  
  Alternativ können Sie auch auf **▶ Run** oben in der Werkzeugleiste klicken. 
- Über **Run → Run all cells** werden alle Zellen auf einmal ausgeführt. Dies ist die sicherste Wahl. Damit werden alle Zellen neu berechnet und die Ausgaben (Datenexport in eine EXCEL-Datei) erstellt.

---

In [1]:
import numpy as np
import pandas as pd

#================ Eingabe ======================

pop = 100_000    # Normierung, nicht ändern
n = 200_000      # Anzahl der MC-Simulationen
seed = 42        # oder None - für die (Pseudo)-Zufallszahlen

# ---------  Erkrankungsraten ------------------ 

attack_rates = [0.15, 0.30, 0.50, 0.70]

# ---------  Min, Wert von Meltzer, Max --------

tri = dict(
    consult_frac=(0.45, 0.50, 0.55),
    hosp_per_consult=(0.024, 0.027, 0.03),
    death_per_consult=(0.004, 0.007, 0.010),
)

# -- Name der EXCEL-Datei, in der die Daten exportiert werden --
# -- Diese Datei muss sich in diesem Ordner befinden -----------

filename="Pandemieplan_RKS_exakt_MC_export.xlsx"

#================================================

rng = np.random.default_rng(seed)

records = []

for ar in attack_rates:
    cf = rng.triangular(*tri["consult_frac"], size=n)
    hp = rng.triangular(*tri["hosp_per_consult"], size=n)
    dp = rng.triangular(*tri["death_per_consult"], size=n)

    ill = pop * ar
    consult = ill * cf
    hosp = consult * hp
    deaths = consult * dp

    df_ar = pd.DataFrame({
        "attack_rate": ar,
        "ill_total": ill,
        "consult_total": consult,
        "hosp_total": hosp,
        "deaths_total": deaths,
    })

    records.append(df_ar)

# DAS ist das gesuchte Objekt
sim = pd.concat(records, ignore_index=True)


In [2]:
import pandas as pd

pop = 100_000  # oder die Population, die du für sim verwendet hast
norm = 100_000 / pop

# Welche Größen willst du ins Excel schreiben?
value_cols = ["consult_total", "hosp_total", "deaths_total"]
qs = [0.05, 0.50, 0.95]

sim_100k = sim.copy()
for c in value_cols:
    sim_100k[c] = sim_100k[c] * norm

qtab = (
    sim_100k.groupby("attack_rate")[value_cols]
    .quantile(qs)
    .unstack(level=1)
)

qtab.columns = [f"{col}_q{int(q*100):02d}" for col, q in qtab.columns]
qtab = qtab.reset_index()

# Optional: schöner als Prozent ausgeben
qtab["attack_rate_pct"] = (qtab["attack_rate"] * 100).round(0).astype(int)

# Für Bericht/Excel: auf ganze Personen runden
for c in qtab.columns:
    if c not in ("attack_rate", "attack_rate_pct"):
        qtab[c] = qtab[c].round(0).astype(int)

# Spaltenreihenfolge (amtlich lesbar)
qtab = qtab[["attack_rate_pct"] + [c for c in qtab.columns if c not in ("attack_rate", "attack_rate_pct")]]
qtab = qtab.rename(columns={"attack_rate_pct": "Erkrankungsrate_%"})
qtab

,Erkrankungsrate_%,consult_total_q05,consult_total_q50,consult_total_q95,hosp_total_q05,hosp_total_q50,hosp_total_q95,deaths_total_q05,deaths_total_q50,deaths_total_q95
0,15,6988,7500,8013,182,202,223,37,52,68
1,30,13974,15001,16025,365,405,447,74,105,137
2,50,23294,25000,26710,609,674,744,123,175,228
3,70,32606,35002,37394,852,944,1042,172,245,319


In [3]:
def fmt_tri(triple: tuple[float, float, float]) -> str:
    a, m, b = triple
    # deutsches Dezimal-Komma:
    def dc(x: float) -> str:
        s = f"{x:.3f}".rstrip("0").rstrip(".")
        return s.replace(".", ",")
    return f"min={dc(a)} · mode={dc(m)} · max={dc(b)}"

def fmt_attack_rates(attack_rates: list[float]) -> str:
    # z. B. [0.15,0.30,0.50,0.70] -> "15 %, 30 %, 50 %, 70 %"
    return ", ".join(f"{int(round(ar*100))} %" for ar in attack_rates)


In [4]:
import pandas as pd
from pathlib import Path
from datetime import date

meta = pd.DataFrame({
    "Schlüssel": [
        "Modell",
        "Simulationstyp",
        "Stichprobengröße n",
        "Zufallsverteilungen",
        "consult_frac",
        "hosp_per_consult",
        "death_per_consult",
        "Erkrankungsraten",
        "Normierung",
        "Rundung",
        "Zufallsseed",
        "Erstellungsdatum",
    ],
    "Wert": [
        "Meltzer (vereinfachtes Kettenmodell)",
        "Monte-Carlo",
        f"{n:,}".replace(",", "."),  # deutsch: 200.000
        "Dreiecksverteilungen",
        fmt_tri(tri["consult_frac"]),
        fmt_tri(tri["hosp_per_consult"]),
        fmt_tri(tri["death_per_consult"]),
        fmt_attack_rates(attack_rates),
        "pro 100.000 Einwohner",
        "intern reell, Darstellung gerundet",
        seed,
        date.today().isoformat(),
    ],
})

excel_path = Path(filename)

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    meta.to_excel(writer, sheet_name="MC_Metadata", index=False)


In [5]:
from pathlib import Path
import pandas as pd

excel_path = Path(filename)
sheet_name = "MC_Quantile_100k"

with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    qtab.to_excel(writer, sheet_name=sheet_name, index=False)

print("Geschrieben:", excel_path, "->", sheet_name)


Geschrieben: Pandemieplan_RKS_exakt_MC_export.xlsx -> MC_Quantile_100k


In [6]:
from openpyxl import load_workbook

def move_sheet_after(excel_path, sheet_to_move: str, after_sheet: str):
    wb = load_workbook(excel_path)

    if sheet_to_move not in wb.sheetnames:
        raise ValueError(f"Sheet '{sheet_to_move}' nicht gefunden.")
    if after_sheet not in wb.sheetnames:
        raise ValueError(f"Sheet '{after_sheet}' nicht gefunden.")

    ws_move = wb[sheet_to_move]
    after_idx = wb.sheetnames.index(after_sheet)

    # openpyxl: Reihenfolge wird über wb._sheets gesteuert
    wb._sheets.remove(ws_move)
    wb._sheets.insert(after_idx + 1, ws_move)

    wb.save(excel_path)

move_sheet_after(excel_path, "MC_Quantile_100k", "Parameter")
print("Sheet-Reihenfolge angepasst.")


Sheet-Reihenfolge angepasst.
